<img src=https://d1.awsstatic.com/logos/aws-logo-lockups/poweredbyaws/PB_AWS_logo_RGB_REV_SQ.8c88ac215fe4e441dc42865dd6962ed4f444a90d.png width="350">

# <center> Neptune Data Exploration - Lab 1 </center>


# Using Gremlin to Access the Graph

The following tutorial walks you through using Gremlin to add vertices, edges, properties, and more to a Neptune graph, highlights some differences in the Neptune-specific Gremlin implementation.

## Show 10 airports:

In [2]:
%%gremlin

g.V().hasLabel('airport').valueMap().limit(10)

## Show which city airport has the most ORIGIN flights, in this period of time:
- Note: see reference at https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States

In [3]:
%%gremlin

g.E().hasLabel('flight_connection').groupCount().by('ORIGIN_CITY_NAME').order(local).by(values, desc).unfold()

## Show which city airport has the most DESTINATION flights, in this period of time:
- Note: see reference at https://en.wikipedia.org/wiki/List_of_the_busiest_airports_in_the_United_States

In [4]:
%%gremlin

g.E().hasLabel('flight_connection').groupCount().by('DESTINATION_CITY_NAME').order(local).by(values, desc).unfold()

## Show 10 airports with the most outgoing routes, in descending order. 

In [5]:
%%gremlin

g.V().hasLabel('airport').order().by(out().count(),desc).limit(10).project('a','b').by('ORIGIN_CODE').by(out().count())

## Show airport connections, that have a distance in-between greater than 4800 miles

In [6]:
%%gremlin

g.V().hasLabel('airport').outE().has('DISTANCE',gt(4800)).valueMap('DISTANCE','CARRIER_NAME','ORIGIN_CITY_NAME','DESTINATION_CITY_NAME').path()

## Show Delay performance for Departures, WITHOUT considering attributes; i.e. DISTANCE travelled by Airline
- Better analysis: for every 100 miles travelled (mileage), how many +15 MIN delays occured?

In [8]:
%%gremlin

g.V().hasLabel('airport').outE().group().by('CARRIER_NAME').by(values('DEPARTURE_DELAY_15MIN').sum()).unfold()

In [9]:
%%gremlin -p v,inv

g.V().hasLabel('airport').out().path().limit(5)

## Finding all routes from Orlando International Airport (MCO)

In [10]:
%%gremlin -p v,inv

g.V().hasLabel('airport').has('ORIGIN_CODE','MCO').out('flight_connection').path().by('ORIGIN_CODE')

In [11]:
my_node_labels = '{"airport":"CITY_NAME"}'
my_edge_labels = '{"flight_connection":"DISTANCE"}'

In [12]:
%%gremlin -d $my_node_labels -l 20 -g country -de $my_edge_labels

g.V().has('ORIGIN_CODE','ATL').outE().inV().path().by(elementMap()).limit(100)